In [15]:
import numpy as np
from funzioni import menu,annealing
import time

In [16]:
# Parametri iniziali
start = time.perf_counter()
A,b,scelta_in,autoval = menu()
dim_mat = len(A)
autovett = None 

== MENU PRINCIPALE ==
1️⃣  Inserisci dati manualmente
2️⃣  Genera dati casualmente


Scegli la stima iniziale:
1️⃣  Media autovalori
2️⃣  Limite di Gershgorin


In [17]:
# Costruzione del vettore precisione p
p = np.zeros(b)
p[0] = -1
p[1:] = [2 ** (-i) for i in range(1, b)]

# Costruzione Psx e Pdx
# riempimento di Psx per colonne con pt:
Psx = np.zeros((dim_mat * b, dim_mat))
for j in range(dim_mat) : 
    Psx[j*b : (j+1)*b , j] = p
# riempimento di Pdx per righe con p:
Pdx = np.zeros((dim_mat, dim_mat * b))
for i in range(dim_mat) : 
    Pdx[i,i*b:(i+1)*b] = p

# Parametri locali
num_reads = 100 
sweeps = 2000
tol = 1e-5
maxiter = 500
num_iter=0

# Ciclo iterativo per la ricerca dell'autovalore minimo e di un autovettore associato
prop = True
while prop == True :
    num_iter = num_iter+1
    H = A - autoval * np.eye(dim_mat)
    autoval_old = autoval
    if not np.all(H == 0):
        vett_reale = annealing(dim_mat, b, p, Psx, H, Pdx,num_reads,sweeps)
        if np.linalg.norm(vett_reale) != 0 :
            autovett = vett_reale/ np.linalg.norm(vett_reale)
            autoval = autovett.T @ A @ autovett
            prop = (abs(autoval - autoval_old) > tol) and ((num_iter +1 ) < maxiter)
        else :
            prop = False
    else : 
        prop = False

end = time.perf_counter()

In [ ]:
print("=== Risultati ===")
if autovett is None :  
    print("il programma ha fallito")
else :
    print("Matrice A :")
    col_widths = [max(len(str(row[i])) for row in A) for i in range(len(A[0]))]
    for row in A:
        row_str = " ".join(f"{str(val).rjust(col_widths[i])}" for i, val in enumerate(row))
        print(f"[ {row_str} ]")
    print("Numero di bit : ", b)
    if (scelta_in == '1') :
        print("La scelta iniziale dell'autovalore è stata la media degli autovalori")
    else :
        print("La scelta iniziale dell'autovalore è stato il limite di Gershgorin")
    print("num_reads : ", num_reads)
    print("sweeps : ", sweeps)
    print("Tolleranza nel while : ", tol)
    print("Numero di iterazioni : ", num_iter)
    print("Tempo di esecuzione (s):", end - start)
    [D, V] = np.linalg.eig(A)
    lambda_min = min(D)
    index_min = np.argmin(D) 
    print('Autovalore calcolato : ', autoval)
    print("Autovalore target : ", lambda_min)
    print('Errore sull\'autovalore : ', abs(lambda_min - autoval))
    print("Autovettore calcolato : \n [", " ".join(f"{x}" for x in autovett), "]")
    print("Autovettore target : \n [", " ".join(f"{x}" for x in V[:, index_min]), "]")
    print('Errore sull\'autovettore : ', np.linalg.norm(autovett - V[:, index_min]))

=== Risultati ===
Matrice A :
[ 1.0 2.0 ]
[ 2.0 1.0 ]
Numero di bit :  2
La scelta iniziale dell'autovalore è stata la media degli autovalori
num_reads :  100
sweeps :  2000
Tolleranza nel while :  1e-05
Numero di iterazioni :  3
Tempo di esecuzione (s): 13.202689700061455
Autovalore calcolato :  -0.9999999999999998
Autovalore target :  -0.9999999999999996
Errore sull'autovalore :  2.220446049250313e-16
Autovettore calcolato : 
 [ 0.7071067811865475 -0.7071067811865475 ]
Autovettore target : 
 [ -0.7071067811865475 0.7071067811865476 ]
Errore sull'autovettore :  1.9999999999999998
